In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc



In [247]:
# Reading files

# Read in the data
# 2018
ela_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM ELA.csv"
)
math_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM Math.csv"
)
science_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM SCIENCE.csv"
)




C:\Users\ericr\AppData\Local\Temp\ipykernel_2096\2568918660.py:5: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ela_2018 = pd.read_csv(
C:\Users\ericr\AppData\Local\Temp\ipykernel_2096\2568918660.py:8: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2018 = pd.read_csv(
C:\Users\ericr\AppData\Local\Temp\ipykernel_2096\2568918660.py:11: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  science_2018 = pd.read_csv(


In [248]:
# Checking types
ela_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654694 entries, 0 to 654693
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ENTITY_CD           654694 non-null  int64 
 1   ENTITY_NAME         654694 non-null  object
 2   YEAR                654694 non-null  int64 
 3   ASSESSMENT_NAME     654694 non-null  object
 4   SUBGROUP_NAME       654694 non-null  object
 5   NUM_TESTED          654694 non-null  int64 
 6   NOT_TESTED          654694 non-null  int64 
 7   LEVEL1_COUNT        654694 non-null  object
 8   LEVEL1_%TESTED      654694 non-null  object
 9   LEVEL2_COUNT        654694 non-null  object
 10  LEVEL2_%TESTED      654694 non-null  object
 11  LEVEL3_COUNT        654694 non-null  object
 12  LEVEL3_%TESTED      654694 non-null  object
 13  LEVEL4_COUNT        654694 non-null  object
 14  LEVEL4_%TESTED      654694 non-null  object
 15  LEVEL5_COUNT        304561 non-null  object
 16  LE

In [249]:
# Our columns of interest
Team_9_keep_cols = ["NUM_TESTED", "NUM_PROF", "TOTAL_SCALE_SCORES", "MEAN_SCORE"]
# subgroup_9_keep_cols =  ["English Language Learners", "Non-English Language Learners"]
subgroup_9_keep_cols =  ["ELL", "NON-ELL"]


In [250]:
# Scouting racial groups
ela_2018.SUBGROUP_NAME.unique()

array(['Hispanic or Latino', 'Homeless', 'Male',
       'Non-English Language Learners', 'Not Economically Disadvantaged',
       'American Indian or Alaska Native',
       'Asian or Native Hawaiian/Other Pacific Islander',
       'Black or African American', 'Economically Disadvantaged',
       'English Language Learners', 'Female', 'General Education',
       'Multiracial', 'All Students', 'Not Migrant',
       'Students with Disabilities', 'White', 'Migrant',
       'Small Group Total', 'American Indian or Alaska Native Female',
       'American Indian or Alaska Native Male',
       'Asian or Native Hawaiian/Other Pacific Islander Female',
       'Asian or Native Hawaiian/Other Pacific Islander Male',
       'Black or African American Female',
       'Black or African American Male', 'Hispanic or Latino Female',
       'Hispanic or Latino Male', 'Multiracial Female',
       'Multiracial Male', 'White Female', 'White Male', 'Not Homeless',
       'Not in Foster Care', 'Parent in Arme

In [251]:


def sanity_check_entity_cd(ds):
    
    for i in ds.ENTITY_CD.unique():
        try: 
            int(i)
        except:
            print(i)

In [252]:
def race_df_maker(df : pd.DataFrame, base_cols : list[str], subgroups : list[str], keep_cols : list[str] = [], impute = False) -> pd.DataFrame:
    """Takes a our base dataframe and stacks the subgroups horizontally
    via a merge.
    """
    ell_dict = {
        "English Language Learners": "ELL",
        "Non-English Language Learners": "NON-ELL",
        "English Language Learner": "ELL",
        "Non-English Language Learner": "NON-ELL",
    }
    copy_df = df.copy() # to contain all the core records
    copy_df['SUBGROUP_NAME'] = copy_df['SUBGROUP_NAME'].map(ell_dict)
    print(copy_df['SUBGROUP_NAME'].unique())

    copy_df['Subject'] = copy_df['ASSESSMENT_NAME'].str.extract(r"([A-Za-z]*).*", expand=False)

    copy_df[copy_df.MEAN_SCORE == 's'] = np.nan

    if impute:
        copy_df['MEAN_SCORE'] = copy_df['MEAN_SCORE'].astype(float)

        copy_df["MEAN_SCORE"].fillna(copy_df.groupby(["YEAR", "ENTITY_CD", "Subject"])["MEAN_SCORE"].transform("mean"), inplace=True)

    base_cols.append("Subject")
    
    if keep_cols != []:
        new_cols = keep_cols + base_cols + ["SUBGROUP_NAME"]

    else:

        new_cols = base_cols + ["SUBGROUP_NAME"]

    base_df = copy_df[new_cols].copy() # get the core records

    base_df.drop_duplicates(subset = base_cols, inplace=True, ignore_index=True) # remove duplicates

    core_df = copy_df[new_cols].copy() # get the core records

    # At this point, we have a df custom to merge with Team 9's base data
    # that contains all the schools,
    # But we still need the data by race

    # We will create a dictionary of dataframes

    for group in subgroups:

        # copy to avoid pointer issues
        
        group_df = core_df[core_df["SUBGROUP_NAME"] == group].copy()

        formatted_name = group.replace(" ", "_").lower()

        base_df = base_df.merge(group_df, on=base_cols, how="left", suffixes=("", f"_{formatted_name}"))

    # Removing the subgroup name columns

    ls_subgroup_cols = [col for col in base_df.columns if "subgroup_name" in col.lower()]

    extra_cols = ['NUM_TESTED', 'NUM_PROF', 'TOTAL_SCALE_SCORES', 'MEAN_SCORE']

    base_df.drop(columns=ls_subgroup_cols + extra_cols, inplace=True)
    
    return base_df

In [253]:
# testing the function
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
).keys()

[nan 'NON-ELL' 'ELL']


Index(['ENTITY_CD', 'YEAR', 'ASSESSMENT_NAME', 'Subject', 'NUM_TESTED_ell',
       'NUM_PROF_ell', 'TOTAL_SCALE_SCORES_ell', 'MEAN_SCORE_ell',
       'NUM_TESTED_non-ell', 'NUM_PROF_non-ell', 'TOTAL_SCALE_SCORES_non-ell',
       'MEAN_SCORE_non-ell'],
      dtype='object')

In [254]:
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
).shape

[nan 'NON-ELL' 'ELL']


(36740, 12)

### 2018

In [255]:
# testing the function
for ds in [ela_2018, math_2018, science_2018]:
    sanity_check_entity_cd(ds)
    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)

[nan 'NON-ELL' 'ELL']
[nan 'ELL' 'NON-ELL']
[nan 'NON-ELL' 'ELL']


In [256]:
final_ds = None

for ds in [ela_2018, math_2018, science_2018]:
    assert 2018 in ds.YEAR.unique()
    print(ds.YEAR.unique())

    # new_ds = race_df_maker(
    #     ds,
    #     ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
    #     subgroup_9_keep_cols,
    #     keep_cols=Team_9_keep_cols,
    # )

    # if final_ds is None:
    #     final_ds = new_ds.copy()

    # else:
    #     final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

[2018 2017]
[2017 2018]
[2018 2017]


In [257]:

# sanity checks

# print("Shape of final_ds:", final_ds.shape)
# print("")

In [258]:
# final_ds.head(10)

In [259]:
# delete the dataframes to save memory
del ela_2018, math_2018, science_2018
gc.collect()

1067

### 2019

In [260]:
# 2019

ela_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM ELA.csv"
)
math_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM MATH.csv"
)
science_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM SCIENCE.csv"
)



# testing the function
for ds in [ela_2019, math_2019, science_2019]:
    sanity_check_entity_cd(ds)


    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)



[nan 'NON-ELL' 'ELL']
[nan 'ELL' 'NON-ELL']
[nan 'ELL' 'NON-ELL']


In [261]:
for ds in [ela_2019, math_2019, science_2019]:

    assert 2019 in ds.YEAR.unique()
    print(ds.YEAR.unique())
    
            
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        subgroup_9_keep_cols,
        keep_cols=Team_9_keep_cols,
    )


    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

[2018 2019]
[nan 'NON-ELL' 'ELL']
[2018 2019]
[nan 'ELL' 'NON-ELL']
[2018 2019]
[nan 'ELL' 'NON-ELL']


In [262]:
ela_2019

,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,SUBGROUP_NAME,NUM_TESTED,NOT_TESTED,LEVEL1_COUNT,LEVEL1_%TESTED,LEVEL2_COUNT,LEVEL2_%TESTED,LEVEL3_COUNT,LEVEL3_%TESTED,LEVEL4_COUNT,LEVEL4_%TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE
0,331400011071,JUAN MOREL CAMPOS SECONDARY SCHOOL,2018,ELA8,Hispanic or Latino,39,6,17,44,15,38,6,15,1,03,7,18,22910,587
1,331400011071,JUAN MOREL CAMPOS SECONDARY SCHOOL,2018,ELA7,Hispanic or Latino,36,2,15,42,15,42,5,14,1,03,6,17,21346,593
2,331400011071,JUAN MOREL CAMPOS SECONDARY SCHOOL,2018,ELA6,Hispanic or Latino,36,2,8,22,13,36,11,31,4,11,15,42,21466,596
3,331400011071,JUAN MOREL CAMPOS SECONDARY SCHOOL,2018,ELA8,Homeless,9,2,5,56,2,22,2,22,0,00,2,22,5284,587
4,331400011071,JUAN MOREL CAMPOS SECONDARY SCHOOL,2018,ELA7,Homeless,6,1,4,67,2,33,0,00,0,00,0,00,3491,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610560,680801040002,DUNDEE ELEMENTARY SCHOOL,2019,ELA4,Students with Disabilities,14,1,11,79,3,21,0,00,0,00,0,00,8074,577
610561,680801040002,DUNDEE ELEMENTARY SCHOOL,2019,ELA5,White,42,1,s,s,s,s,s,s,s,s,s,s,s,s
610562,680801040002,DUNDEE ELEMENTARY SCHOOL,2019,ELA6,White,38,2,11,29,11,29,6,16,10,26,16,42,22787,600
610563,680801040002,DUNDEE ELEMENTARY SCHOOL,2019,ELA3,White,26,3,4,15,10,38,12,46,0,00,12,46,15527,597


In [263]:

# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")

Shape of final_ds: (114398, 12)



In [264]:
final_ds.tail(10)

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_ell,NUM_PROF_ell,TOTAL_SCALE_SCORES_ell,MEAN_SCORE_ell,NUM_TESTED_non-ell,NUM_PROF_non-ell,TOTAL_SCALE_SCORES_non-ell,MEAN_SCORE_non-ell
114388,6.806011e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114389,6.806011e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114390,6.806011e+11,2019.0,Science4,Science,NaN,NaN,NaN,NaN,106.0,99,8979,85
114391,6.808010e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114392,6.808010e+11,2019.0,Science4,Science,NaN,NaN,NaN,NaN,66.0,52,5020,76
114393,6.808010e+11,2019.0,Science8,Science,NaN,NaN,NaN,NaN,39.0,22,2572,66
114394,6.808010e+11,2019.0,Science8,Science,NaN,NaN,NaN,NaN,38.0,22,2531,67
114395,6.808010e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114396,6.808010e+11,2019.0,Science4,Science,NaN,NaN,NaN,NaN,65.0,52,4999,77
114397,6.808010e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [265]:
# delete the dataframes to save memory
del ela_2019, math_2019, science_2019
gc.collect()

116

### 2020 does not exist

### 2021

In [266]:



# 2020 is unobserved

# 2021
ela_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM ELA.csv"
)
math_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM Math.csv"
)
science_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM SCIENCE.csv"
)


# testing the function

for ds in [ela_2021, math_2021, science_2021]:
    
    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(ds.YEAR.unique(), ds.ASSESSMENT_NAME.unique())
        print(e)
        print(ds.head())
        print(ds.shape)
        continue

C:\Users\ericr\AppData\Local\Temp\ipykernel_2096\3238058395.py:7: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2021 = pd.read_csv(


[nan 'NON-ELL' 'ELL']
[2021] ['ELA3' 'ELA4' 'ELA5' 'ELA6' 'ELA7' 'ELA8' 'ELA3_8']
'DataFrame' object has no attribute 'MEAN_SCORE'
   INSTITUTION_ID     ENTITY_CD         ENTITY_NAME  YEAR ASSESSMENT_NAME   
0    8.000001e+11  111111111111  All Public Schools  2021            ELA3  \
1    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
2    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
3    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
4    8.000001e+11  111111111111  All Public Schools  2021            ELA3   

                      SUBGROUP_NAME  TOTAL_COUNT  NOT_TESTED  PCT_NOT_TESTED   
0                      All Students       192560       99642              52  \
1                            Female        93485       47472              51   
2                              Male        99075       52170              53   
3  American Indian or Alaska Native         1469         963              66   
4     

In [267]:
math_2021.keys()

Index(['INSTITUTION_ID', 'ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'ASSESSMENT_NAME',
       'SUBGROUP_NAME', 'TOTAL_COUNT', 'NOT_TESTED', 'PCT_NOT_TESTED',
       'NUM_TESTED', 'PCT_TESTED', 'LEVEL1_COUNT', 'LEVEL1_%TESTED',
       'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT', 'LEVEL3_%TESTED',
       'LEVEL4_COUNT', 'LEVEL4_%TESTED', 'LEVEL5_COUNT', 'LEVEL5_%TESTED',
       'NUM_PROF', 'PER_PROF', 'TOTAL_EXEMPT', 'NUM_EXEMPT_NTEST',
       'PCT_EXEMPT_NTEST', 'NUM_EXEMPT_TEST', 'PCT_EXEMPT_TEST'],
      dtype='object')

>2021 has no mean scores. The message propagates across all 3 subjects. 

In [268]:
# delete the dataframes to save memory
del ela_2021, math_2021, science_2021
gc.collect()

66

### 2022

In [269]:


# 2022

ela_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM ELA.csv"
)
math_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM Math.csv"
)
science_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM SCIENCE.csv"
)

# testing the function
for ds in [ela_2022, math_2022, science_2022]:

    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)

C:\Users\ericr\AppData\Local\Temp\ipykernel_2096\328423560.py:6: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2022 = pd.read_csv(


[nan 'NON-ELL' 'ELL']
[nan 'NON-ELL' 'ELL']
[nan 'NON-ELL' 'ELL']


In [270]:
for ds in [ela_2022, math_2022, science_2022]:

    assert 2022 in ds.YEAR.unique()
    print(ds.YEAR.unique())
    
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        subgroup_9_keep_cols,
        keep_cols=Team_9_keep_cols,
    )

    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

[2021 2022]
[nan 'NON-ELL' 'ELL']
[2021 2022]
[nan 'NON-ELL' 'ELL']
[2021 2022]
[nan 'NON-ELL' 'ELL']


In [271]:
# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")
final_ds.tail(10)

Shape of final_ds: (243735, 12)



,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_ell,NUM_PROF_ell,TOTAL_SCALE_SCORES_ell,MEAN_SCORE_ell,NUM_TESTED_non-ell,NUM_PROF_non-ell,TOTAL_SCALE_SCORES_non-ell,MEAN_SCORE_non-ell
243725,6.806011e+11,2022.0,Science4,Science,NaN,NaN,NaN,NaN,79.0,72,6589,83
243726,6.808010e+11,2022.0,Science8,Science,NaN,NaN,NaN,NaN,40.0,23,2767,69
243727,6.808010e+11,2022.0,CombinedScience,CombinedScience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243728,6.808010e+11,2022.0,Science4,Science,NaN,NaN,NaN,NaN,50.0,46,3968,79
243729,6.808010e+11,2022.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243730,6.808010e+11,2022.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243731,6.808010e+11,2022.0,CombinedScience,CombinedScience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243732,6.808010e+11,2022.0,Science8,Science,NaN,NaN,NaN,NaN,39.0,23,2711,70
243733,6.808010e+11,2022.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243734,6.808010e+11,2022.0,Science4,Science,NaN,NaN,NaN,NaN,49.0,45,3903,80


In [272]:
# delete the dataframes to save memory
del ela_2022, math_2022, science_2022

### Merging with Team 9's All Data CSV

In [273]:
all_data = pd.read_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\final.csv")

In [274]:
# sanity checks
print("Shape of all_data:", all_data.shape)

Shape of all_data: (2933498, 4)


In [275]:

all_data.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME
0,1,2018,ELA,ELA5
1,1,2018,ELA,ELA3_8
2,1,2018,ELA,ELA3
3,1,2018,ELA,ELA7
4,1,2018,ELA,ELA8


In [276]:
full_data = pd.merge(all_data, final_ds, how="left", on=["ENTITY_CD", "YEAR", "Subject", "ASSESSMENT_NAME"])
full_data.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME,NUM_TESTED_ell,NUM_PROF_ell,TOTAL_SCALE_SCORES_ell,MEAN_SCORE_ell,NUM_TESTED_non-ell,NUM_PROF_non-ell,TOTAL_SCALE_SCORES_non-ell,MEAN_SCORE_non-ell
0,1,2018,ELA,ELA5,8330.0,262,4828302,580,60109.0,25728,36282020,604
1,1,2018,ELA,ELA3_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2018,ELA,ELA3,11825.0,2412,6932450,586,55412.0,31624,33429108,603
3,1,2018,ELA,ELA7,7019.0,203,4061768,579,58231.0,27609,35193088,604
4,1,2018,ELA,ELA8,6704.0,315,3872714,578,56479.0,31727,34146979,605


In [277]:
full_data.keys()

Index(['ENTITY_CD', 'YEAR', 'Subject', 'ASSESSMENT_NAME', 'NUM_TESTED_ell',
       'NUM_PROF_ell', 'TOTAL_SCALE_SCORES_ell', 'MEAN_SCORE_ell',
       'NUM_TESTED_non-ell', 'NUM_PROF_non-ell', 'TOTAL_SCALE_SCORES_non-ell',
       'MEAN_SCORE_non-ell'],
      dtype='object')

In [278]:
full_data.to_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\full_data_ELL_FIXED3.csv", index=False)

In [279]:
full_data.isnull().sum()

na_cols = full_data.isnull().sum()[full_data.isnull().sum() > 0].index

In [280]:
full_data.isnull().sum()


ENTITY_CD                           0
YEAR                                0
Subject                             0
ASSESSMENT_NAME                     0
NUM_TESTED_ell                2170713
NUM_PROF_ell                  2170713
TOTAL_SCALE_SCORES_ell        2263157
MEAN_SCORE_ell                2263157
NUM_TESTED_non-ell            1584134
NUM_PROF_non-ell              1584134
TOTAL_SCALE_SCORES_non-ell    1619648
MEAN_SCORE_non-ell            1619648
dtype: int64

In [281]:
older_copy = pd.read_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\full_data_ELL_FIXED2.csv")

In [282]:
older_copy.isnull().sum()

ENTITY_CD                                                 0
YEAR                                                      0
Subject                                                   0
ASSESSMENT_NAME                                           0
NUM_TESTED_english_language_learners                2170713
NUM_PROF_english_language_learners                  2170713
TOTAL_SCALE_SCORES_english_language_learners        2263157
MEAN_SCORE_english_language_learners                2263157
NUM_TESTED_non-english_language_learners            1584134
NUM_PROF_non-english_language_learners              1584134
TOTAL_SCALE_SCORES_non-english_language_learners    1619648
MEAN_SCORE_non-english_language_learners            1619648
dtype: int64

In [283]:
test_copy = full_data.copy()

In [284]:

# for col in na_cols:

#     test_copy[col].fillna(test_copy.groupby(["YEAR","ENTITY_CD","Subject", "ASSESSMENT_NAME"])[col].transform("median"), inplace=True)

In [285]:
test_copy.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME,NUM_TESTED_ell,NUM_PROF_ell,TOTAL_SCALE_SCORES_ell,MEAN_SCORE_ell,NUM_TESTED_non-ell,NUM_PROF_non-ell,TOTAL_SCALE_SCORES_non-ell,MEAN_SCORE_non-ell
0,1,2018,ELA,ELA5,8330.0,262,4828302,580,60109.0,25728,36282020,604
1,1,2018,ELA,ELA3_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2018,ELA,ELA3,11825.0,2412,6932450,586,55412.0,31624,33429108,603
3,1,2018,ELA,ELA7,7019.0,203,4061768,579,58231.0,27609,35193088,604
4,1,2018,ELA,ELA8,6704.0,315,3872714,578,56479.0,31727,34146979,605


In [286]:
test_copy.isnull().sum()

ENTITY_CD                           0
YEAR                                0
Subject                             0
ASSESSMENT_NAME                     0
NUM_TESTED_ell                2170713
NUM_PROF_ell                  2170713
TOTAL_SCALE_SCORES_ell        2263157
MEAN_SCORE_ell                2263157
NUM_TESTED_non-ell            1584134
NUM_PROF_non-ell              1584134
TOTAL_SCALE_SCORES_non-ell    1619648
MEAN_SCORE_non-ell            1619648
dtype: int64

In [287]:
full_data

full_data["MEAN_SCORE"].fillna(copy_df.groupby(["YEAR", "ENTITY_CD", "Subject"])["MEAN_SCORE"].transform("mean"), inplace=True)


KeyError: 'MEAN_SCORE'